In [1]:
from pprint import pprint # used for pretty printing
import requests # used to get data from the a URL
import pandas as pd # used to analyse the results
from modlamp.descriptors import PeptideDescriptor, GlobalDescriptor 
search_url = "https://www.ebi.ac.uk/pdbe/search/pdb/select?" # the rest of the URL used for PDBe's search API.

In [2]:
data_raw=pd.read_csv('grampa_AA.csv', index_col=0)
#Erase sequences with unusual modifications
data_raw=data_raw[data_raw["has_unusual_modification"]==False]
# Drop irrelevant columns
datos_raw=data_raw.drop(['url_source','datasource_has_modifications','database','has_unusual_modification'], axis='columns')

In [3]:
sec_raw=datos_raw["sequence"]

sec_0=[]
[sec_0.append(x) for x in sec_raw if x not in sec_0]
len(sec_0)

6169

In [5]:
#Creamos una lista con las secuencias del DF para poder trabajarla en modlAMP 
sec_raw=pd.Series.tolist(data_raw["sequence"])
#Calcular la longitud de las secuencias
globaldesc=GlobalDescriptor(sec_raw)
globaldesc.length()
lon=globaldesc.descriptor
data_raw["sequence_length"]=lon

In [6]:
test_data=data_raw[(data_raw["sequence_length"]>=10)].copy()

In [7]:
sec_prueba=test_data["sequence"]
sec_1=[]
[sec_1.append(x) for x in sec_prueba if x not in sec_1]
len(sec_1)

5638

We will now define some functions which will use to get the data from PDBe's search API

In [2]:
def make_request_post(search_dict, number_of_rows=10):
    """
    makes a post request to the PDBe API
    :param dict search_dict: the terms used to search
    :param number_of_rows: number or rows to return - initially limited to 10
    :return dict: response JSON
    """
    # make sure we get the number of rows we need
    if 'rows' not in search_dict:
        search_dict['rows'] = number_of_rows
    # set the return type to JSON
    search_dict['wt'] = 'json'

    # do the query
    response = requests.post(search_url, data=search_dict)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))

    return {}

def format_sequence_search_terms(sequence, filter_terms=None):
    """
    Format parameters for a sequence search
    :param str sequence: one letter sequence
    :param lst filter_terms: Terms to filter the results by
    :return str: search string
    """
    # first we set the parameters which we will pass to PDBe's search
    params = {
        'json.nl': 'map',
        'start': '0',
        'sort': 'fasta(e_value) asc',
        'xjoin_fasta': 'true',
        'bf': 'fasta(percentIdentity)',
        'xjoin_fasta.external.expupperlim': '0.1',
        'xjoin_fasta.external.sequence': sequence,
        'q': '*:*',
        'fq': '{!xjoin}xjoin_fasta'
    }
    # we make sure that we add required filter terms if they aren't present
    if filter_terms:
        for term in ['pdb_id', 'entity_id', 'entry_entity', 'chain_id']:
            filter_terms.append(term)
        filter_terms = list(set(filter_terms))
        params['fl'] = ','.join(filter_terms)

    # returns the parameter dictionary
    return params


def run_sequence_search(sequence, filter_terms=None, number_of_rows=10):
    """
    Runs a sequence search and results the results
    :param str sequence: sequence in one letter code
    :param lst filter_terms: terms to filter the results by
    :param int number_of_rows: number of results to return
    :return lst: List of results
    """
    search_dict = format_sequence_search_terms(sequence=sequence, filter_terms=filter_terms)
    response = make_request_post(search_dict=search_dict, number_of_rows=number_of_rows)
    results = response.get('response', {}).get('docs', [])
    print('Number of results {}'.format(len(results)))

    # we now have to go through the FASTA results and join them with the main results

    raw_fasta_results = response.get('xjoin_fasta').get('external')
    fasta_results = {} # results from FASTA will be stored here - key'd by PDB ID and Chain ID

    # go through each FASTA result and get the E value, percentage identity and sequence from the result

    for fasta_row in raw_fasta_results:
        # join_id = fasta_row.get('joinId')
        fasta_doc = fasta_row.get('doc', {})
        percent_identity = fasta_doc.get('percent_identity')
        e_value = fasta_doc.get('e_value')
        return_sequence = fasta_row.get('return_sequence_string')
        pdb_id_chain = fasta_doc.get('pdb_id_chain').split('_')
        pdb_id = pdb_id_chain[0].lower()
        chain_id = pdb_id_chain[-1]
        join_id = '{}_{}'.format(pdb_id, chain_id)
        fasta_results[join_id] = {'e_value': e_value,
                                  'percentage_identity': percent_identity,
                                  'return_sequence': return_sequence}

    # now we go through the main results and add the FASTA results
    ret = [] # final results will be stored here.
    for row in results:
        pdb_id = row.get('pdb_id').lower()
        chain_ids = row.get('chain_id')
        for chain_id in chain_ids:
            search_id = '{}_{}'.format(pdb_id, chain_id)
            entry_fasta_results = fasta_results.get(search_id, {})
            # we will only keep results that match the search ID
            if entry_fasta_results:
                row['e_value'] = entry_fasta_results.get('e_value')
                row['percentage_identity'] = entry_fasta_results.get('percentage_identity')
                row['result_sequence'] = entry_fasta_results.get('return_sequence_string')

                ret.append(row)
    return ret

In [3]:
def multiple_sequence_search(sequences):
    """
    Performs sequence search for multiple entries
    :param lst sequences: a list coontaining all the query sequences
    """
    results=dict.fromkeys(sequences)
    i=0
    while i < len(sequences):
        for values in results:
            results[values]=run_sequence_search(sequences[i], filter_terms=['pdb_id', 'molecule_name'])
            i=i+1
            print(str(i)+" sequences of "+str(len(sequences)))
        
    return results

In [4]:
def results_to_dict(results):
    """
    Delete sequences with no matches inside results dictionary
    :param dict results: Dictionary containing all the results from multiple sequence search
    """
    new_dict = {k: v for k, v in results.items() if len(v) != 0 }
    return new_dict

In [ ]:
res = multiple_sequence_search(sec_1)

In [ ]:
dict_res = results_to_dict(res)